# Venture Funding with Deep Learning

Alphabet Soup venture capital needs assistance to allocate funds. Its business team receives many funding applications from startups every day. This team needs to create a model that predicts whether applicants will be successful if funded by Alphabet Soup.

The business team provided a CSV containing more than 34,000 organizations that have received funding from Alphabet Soup over the years. Using machine learning and neural networks, we create a binary classifier model that will predict whether an applicant will become a successful business. The CSV file contains a variety of information about these businesses, including whether or not they ultimately became successful.

## Steps:

The steps for this analysis are broken out into the following sections:

* Prepare the data for use on a neural network model.

* Compile and evaluate a binary classification model using a neural network.

* Optimize the neural network model.


### Prepare the Data for Use on a Neural Network Model 

Using Pandas and scikit-learn’s `StandardScaler()`, we preprocess the dataset so that we can use it to compile and evaluate the neural network model later.

The following are the data preparation steps:

1. Read the `applicants_data.csv` file into a Pandas DataFrame. Locate the categorical variables that will need to be encoded, as well as columns that may define our features and target variables.   

2. Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.
 
3. Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

4. Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables, using the Pandas `concat()` function.

5. Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset will be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns will define the features dataset. 

6. Split the features and target sets into training and testing datasets.

7. Use scikit-learn's `StandardScaler` to scale the features data.

### Compile and Evaluate a Binary Classification Model Using a Neural Network

Use TensorFlow to design a binary classification deep neural network model. This model should use the dataset’s features to predict whether an Alphabet Soup&ndash;funded startup will be successful based on the features in the dataset. We consider the number of inputs to determine the number of layers that our model will contain or the number of neurons on each layer. Then, we compile and fit your model. Finally, we evaluate our binary classification model to calculate the model’s loss and accuracy. 
 
To do so, we complete the following steps:

1. Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras. We start with a two-layer deep neural network model that uses the `relu` activation function for both layers.

2. Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric. When fitting the model, we start with a small number of epochs, such as 20, 50, or 100.

3. Evaluate the model using the test data to determine the model’s loss and accuracy.

4. Save and export our model to an HDF5 file, and name the file `AlphabetSoup.h5`. 

### Optimize the Neural Network Model

Using TensorFlow and Keras, we optimize our model to improve the model's accuracy. We try changing several parameters, that not always imply a better accuracy, but help us in the route to optimize the model. We have make copies of the starter notebook in the same folder, rename them, and code each model optimization in a new notebook. We did 11 attempts  in total.

To optimize our model for a predictive accuracy as close to 1 as possible, we use any or all of the following techniques:

> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.

We display the accuracy scores achieved by each model, and compare the results, and save each of your models as an HDF5 file.

### Conclusions

At the end of this notebook, you will find a table with results, and our conclusions.


In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from datetime import date

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  


In [2]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(
    Path('Resources/applicants_data.csv',)
)

# Review the DataFrame
applicant_data_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Review the data types associated with the columns
applicant_data_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [4]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN','NAME'])

# Review the DataFrame
applicant_data_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [5]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes=='object'].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [7]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [8]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
encoded_df.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [9]:
non_categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes!='object'].index)
non_categorical_variables

['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL']

In [10]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df =pd.concat([applicant_data_df[non_categorical_variables], encoded_df], axis=1)

# Review the Dataframe
encoded_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [11]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y.head()


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [12]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=['IS_SUCCESSFUL'])

# Review the features DataFrame
X.head()


,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 6: Split the features and target sets into training and testing datasets.


In [13]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y)


### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [15]:
# Define the the number of inputs (features) to the model
# This is the numbers of columns in the X array
number_input_features = X.shape[1]

# Review the number of features
number_input_features


116

In [16]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [17]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = np.ceil(np.sqrt(number_input_features * number_output_neurons))

# Review the number hidden nodes in the first layer
hidden_nodes_layer1


11.0

In [18]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  np.ceil(np.sqrt(hidden_nodes_layer1 * number_output_neurons))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


4.0

In [19]:
# Create the Sequential model instance
nn = Sequential()


In [20]:
# Add the first hidden layer
nn.add(
    Dense(
        units=hidden_nodes_layer1,
        activation='relu',
        input_dim=number_input_features
    )
)


In [21]:
# Add the second hidden layer
nn.add(
    Dense(
        units=hidden_nodes_layer2,
        activation='relu'
    )
)


In [22]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(
    Dense(
        units=1,
        activation='sigmoid'
    )
)


In [23]:
# Display the Sequential model summary
nn.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 11)                1287      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 48        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 1,340
Trainable params: 1,340
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [24]:
# Compile the Sequential model
nn.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [25]:
# Fit the model using 50 epochs and the training data
# nn_model will take the history
nn_model=nn.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6389 - accuracy: 0.6887
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5764 - accuracy: 0.7240
Epoch 3/50
804/804 [==============================] - 1s 964us/step - loss: 0.5614 - accuracy: 0.7292
Epoch 4/50
804/804 [==============================] - 1s 992us/step - loss: 0.5556 - accuracy: 0.7298
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5519 - accuracy: 0.7287
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5505 - accuracy: 0.7308
Epoch 7/50
804/804 [==============================] - 1s 989us/step - loss: 0.5493 - accuracy: 0.7321
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7324
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7331
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [26]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(
                                        X_test_scaled,
                                        y_test,
                                        verbose=2
)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5524 - accuracy: 0.7287
Loss: 0.5524305105209351, Accuracy: 0.7287463545799255


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 


In [27]:
# Set the model's file path
file_path = Path('Resources/AlphabetSoup.h5')

# Export your model to a HDF5 file
nn.save(file_path)


In [74]:
# Display the model loss and accuracy results
results_df=pd.DataFrame(
                    data={
                        'Date': [date.today()],
                        'Model'   : [0],
                        'Observations': ['Original model, geometric mean for number of layers'],
                        'Loss'    : [model_loss], 
                        'Accuracy': [model_accuracy],
                        'Hidden layers': 2,
                        'Input Features':[number_input_features],
                        'Output Neurons': [number_output_neurons],
                        'Neurons in Hidden Layers1' : [hidden_nodes_layer1],
                        'Neurons in Hidden Layers2' : [hidden_nodes_layer2],
                        'Epochs'         : [50],
                        'Activation Hidden layer 1': ['relu'],
                        'Activation Hidden layer 2': ['relu'],
                        'Activation Output layer 1': ['sigmoid'],
                        'loss':['binary_crossentropy'],
                        'optimizer':['adam'],
                        'metrics':['accuracy']          
                    })

print(results_df.T)

                                                                           0
Date                                                              2021-09-26
Model                                                                      0
Observations               Original model, geometric mean for number of l...
Loss                                                                0.552431
Accuracy                                                            0.728746
Hidden layers                                                              2
Input Features                                                           116
Output Neurons                                                             1
Neurons in Hidden Layers1                                               11.0
Neurons in Hidden Layers2                                                4.0
Epochs                                                                    50
Activation Hidden layer 1                                               relu

In [76]:
results_df.to_csv('Resources/results.csv', mode='a', header=True) 

In [95]:
results_df=[]

---

## Optimize the neural network models
In separate files there are 11 alternative neural networks models for the classification. Some of them did better, and some of them did worst. Comparative results are at the end of this file.


### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

## RESULTS



In [96]:
# Preparing data colected from the different models in a csv file
results_df=pd.read_csv('Resources/results_final.csv', index_col='Model')


In [97]:
# This we kept in case we change headers and use header=True in the models in future uses
results_df=results_df[results_df.index!='Model'] 

results_df.drop(columns='Unnamed: 0', inplace=True)
results_df.sort_values(by='Accuracy', ascending=False, inplace=True)

In [98]:
results_df.head(3)

,Date,Observations,Loss,Accuracy,Hidden layers,Input Features,Output Neurons,Neurons in Hidden Layers1,Neurons in Hidden Layers2,Epochs,Activation Hidden layer 1,Activation Hidden layer 2,Activation Output layer 1,loss,optimizer,metrics
Model,,,,,,,,,,,,,,,,
Observations,Model,Loss,Accuracy,Hidden layers,Input Features,Output Neurons,Neurons in Hidden Layers1,Neurons in Hidden Layers2,Epochs,Activation Hidden layer 1,Activation Hidden layer 2,Activation Output layer 1,loss,optimizer,metrics,NaN
11,9/26/21,"Recoded data, in between geometric/arithmetic,...",0.54547137,0.740174949,2,108,1,43,20,50,relu,relu,sigmoid,binary_crossentropy,adam,accuracy
7,9/26/21,"Recoded data, geometric, MinMaxScaler",0.547604859,0.736676395,2,108,1,11,4,50,relu,relu,sigmoid,binary_crossentropy,adam,accuracy


# Comparative Results

Several adjustements where applied to the original model, to see the effect. None of them generated a significant imporvement. Probably the data is not enough, and we will need to add more features in order to improve the model. Several of the chnges did not provoque an improvement.

The changes tried were:

1) Changing the number of nodes: using aritmetic means, geometric means, and in between. The best results were in between.

2) Changing the amount of layers. Adding a third layer did not generate much effect.

3) Changing the Standarization of the data for Normalization. This provoque a positive impact. Nevertheless, very small (around 1% better)

4) Recoding the data. Particularly recoding the Income Amount columns. It did generate a little improvement. Also, encoding using get_dummies instead of the OneHotEncoder.

5) Changing the activiation from 'relu' to 'selu'. 'relu' showed to have better results consistently.

Results can be seen in the table below, sorted by best accuracy. Model 0 is the original, and it's in the middle of them. The 'Observations' column give a summary keyword of the changes. However, for details, the models are included in separate files, and they present the details at the top and in the code.

In [99]:
with pd.option_context('display.max_colwidth', 600):
    display(results_df[['Date', 'Observations', 'Loss', 'Accuracy', 'Hidden layers', 'Epochs']])


,Date,Observations,Loss,Accuracy,Hidden layers,Epochs
Model,,,,,,
Observations,Model,Loss,Accuracy,Hidden layers,Input Features,Activation Hidden layer 1
11,9/26/21,"Recoded data, in between geometric/arithmetic, MinMaxScaler",0.54547137,0.740174949,2,50
7,9/26/21,"Recoded data, geometric, MinMaxScaler",0.547604859,0.736676395,2,50
10,9/26/21,"Recoded data, geometric/aritmetic, MinMaxScaler",0.548608422,0.735043705,2,50
2,9/26/21,"Adding a hidden layer, increasing epochs",0.557831407,0.730845451,3,80
5,9/25/21,"Recoded data keeping 0, arithmetic",0.550725102,0.730029166,2,50
8,9/25/21,"Recoded data, arithmetic, MinMaxScaler",0.553575158,0.729912519,2,50
6,9/26/21,"Original model, MinMaxScaler",0.548836827,0.728979588,2,50
0,9/26/21,"Original model, geometric mean for number of layers",0.552430511,0.728746355,2,50


# Conclusions

1) The use of normalization or standarization makes a difference when modelling. In this case, using MinMaxScaler provided better results than the StandardScaler

2) The activation functions selected make a difference in the accuracy of the model. In our case, `relu` provided the better results.

3) Adding a layer didn't make much impact to improve the original model. However, playing with the number of nodes does makes a difference.

4) The way of recoding the data and changing the way to code the dummies made an impact. For example, when recoding the Income Amount from classes to number, it was better to set the missing values as zero than as the median.

Overall, it was very hard to improve the model. It may be that the data does not contain much more information that we can extract with a neural network model.